In [1]:
from demag_tensor import *
from time import time
import numpy as np

def format_time(_time):
    v = round(_time, 3)
    return f"{v//3600}h {(v % 3600)//60}m {round(v % 60, 3)}s"

In [11]:
times = []
for i in range(1, 26):
    start = time()
    calculate_demag_tensor((100, i, 1), (1e-9, 1e-9, 1e-9))
    times.append(start)
    time_taken = time() - start
    print(f"Cells : {100 * i} ; time taken : {format_time(time_taken)}")

Calculating the demagnetization tensor


/home/ache/Desktop/PROJS/micromagnetics/prototyping/demagnatization_tensor/demag_tensor.py:26: RuntimeWarning: invalid value encountered in double_scalars
  - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
/home/ache/Desktop/PROJS/micromagnetics/prototyping/demagnatization_tensor/demag_tensor.py:27: RuntimeWarning: invalid value encountered in double_scalars
  - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \


(199, 1, 1, 6)
Cells : 100 ; time taken : 0.0h 0.0m 1.202s
Calculating the demagnetization tensor
(199, 3, 1, 6)
Cells : 200 ; time taken : 0.0h 0.0m 3.438s
Calculating the demagnetization tensor
(199, 5, 1, 6)
Cells : 300 ; time taken : 0.0h 0.0m 5.641s
Calculating the demagnetization tensor
(199, 7, 1, 6)
Cells : 400 ; time taken : 0.0h 0.0m 8.026s
Calculating the demagnetization tensor
(199, 9, 1, 6)
Cells : 500 ; time taken : 0.0h 0.0m 10.235s
Calculating the demagnetization tensor
(199, 11, 1, 6)
Cells : 600 ; time taken : 0.0h 0.0m 12.559s
Calculating the demagnetization tensor
(199, 13, 1, 6)
Cells : 700 ; time taken : 0.0h 0.0m 14.877s
Calculating the demagnetization tensor
(199, 15, 1, 6)
Cells : 800 ; time taken : 0.0h 0.0m 17.029s
Calculating the demagnetization tensor
(199, 17, 1, 6)
Cells : 900 ; time taken : 0.0h 0.0m 19.298s
Calculating the demagnetization tensor
(199, 19, 1, 6)
Cells : 1000 ; time taken : 0.0h 0.0m 21.538s
Calculating the demagnetization tensor
(199, 21

In [2]:
# references : 
# https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1029/93JB00694
# https://arxiv.org/pdf/1411.7188.pdf
# 
#
# fidimag : compute_demag_tensors_2dpbc

import numpy as np
from math import asinh, atan, sqrt, pi

# a very small number
eps = 1e-15
n     = (100, 25, 1)
dx    = (5e-9, 5e-9, 3e-9)
demag_tensor_file = "demag_tensor.npy"

# newell f
def _f(p):
  #print(type(p))
  x, y, z = abs(p[0]), abs(p[1]), abs(p[2])
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
def _g(p):
  x, y, z = p[0], p[1], abs(p[2])
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0


# demag tensor setup
def set_n_demag(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      value += (-1)**sum(i) * func(list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute)))
    it[0] = - value / (4 * pi * np.prod(dx))
    it.iternext()

def identity(p):
    return 0

def calculate_demag_tensor(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])

    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    np.save(demag_tensor_file, n_demag)

In [8]:
n = (125, 25, 1)
dx = (1e-9, 1e-9, 1e-9)
t = time()

f = identity
g = identity

ts = calculate_demag_tensor(n ,dx)
print(f"Took {time() - t} s")

Calculating the demagnetization tensor
Took 47.010547161102295 s


In [9]:
f = _f
g = _g
ts = calculate_demag_tensor(n, dx)
print(f"Took {time() - t} s")

Calculating the demagnetization tensor
Took 117.80557084083557 s


In [3]:
for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
    print(i)

[0 0 0 0 0 0]
[0 0 0 0 0 1]
[0 0 0 0 1 0]
[0 0 0 0 1 1]
[0 0 0 1 0 0]
[0 0 0 1 0 1]
[0 0 0 1 1 0]
[0 0 0 1 1 1]
[0 0 1 0 0 0]
[0 0 1 0 0 1]
[0 0 1 0 1 0]
[0 0 1 0 1 1]
[0 0 1 1 0 0]
[0 0 1 1 0 1]
[0 0 1 1 1 0]
[0 0 1 1 1 1]
[0 1 0 0 0 0]
[0 1 0 0 0 1]
[0 1 0 0 1 0]
[0 1 0 0 1 1]
[0 1 0 1 0 0]
[0 1 0 1 0 1]
[0 1 0 1 1 0]
[0 1 0 1 1 1]
[0 1 1 0 0 0]
[0 1 1 0 0 1]
[0 1 1 0 1 0]
[0 1 1 0 1 1]
[0 1 1 1 0 0]
[0 1 1 1 0 1]
[0 1 1 1 1 0]
[0 1 1 1 1 1]
[1 0 0 0 0 0]
[1 0 0 0 0 1]
[1 0 0 0 1 0]
[1 0 0 0 1 1]
[1 0 0 1 0 0]
[1 0 0 1 0 1]
[1 0 0 1 1 0]
[1 0 0 1 1 1]
[1 0 1 0 0 0]
[1 0 1 0 0 1]
[1 0 1 0 1 0]
[1 0 1 0 1 1]
[1 0 1 1 0 0]
[1 0 1 1 0 1]
[1 0 1 1 1 0]
[1 0 1 1 1 1]
[1 1 0 0 0 0]
[1 1 0 0 0 1]
[1 1 0 0 1 0]
[1 1 0 0 1 1]
[1 1 0 1 0 0]
[1 1 0 1 0 1]
[1 1 0 1 1 0]
[1 1 0 1 1 1]
[1 1 1 0 0 0]
[1 1 1 0 0 1]
[1 1 1 0 1 0]
[1 1 1 0 1 1]
[1 1 1 1 0 0]
[1 1 1 1 0 1]
[1 1 1 1 1 0]
[1 1 1 1 1 1]
